In [1]:
from bs4 import BeautifulSoup
import requests as rt
import pandas as pd

In [2]:
url = 'https://josaa.admissions.nic.in/applicant/seatmatrix/OpeningClosingRankArchieve.aspx'
params = {
    #"ctl00$ContentPlaceHolder1$ddlroundno": 
    "ctl00$ContentPlaceHolder1$ddlInstype": "ALL",
    "ctl00$ContentPlaceHolder1$ddlInstitute": "ALL",
    "ctl00$ContentPlaceHolder1$ddlBranch": "ALL",
    "ctl00$ContentPlaceHolder1$ddlSeatType": "ALL",
    "ctl00$ContentPlaceHolder1$btnSubmit": "Submit"
}

In [3]:
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
rounds = ['1', '2', '3', '4', '5', '6', '7']

In [4]:
def scrape(Year, Round):   
    with rt.Session() as s:
        R = s.get(url)
        data = {}

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlYear"] = Year
        R = s.post(url, data=data)

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlroundno"] = Round
        R = s.post(url, data=data)

        for key, value in params.items():
            data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
            data[key] = value
            R = s.post(url, data=data)
        #print(R.text)
        #print(data)

    table = BeautifulSoup(R.text, 'lxml').find(id = 'ctl00_ContentPlaceHolder1_GridView1')
    df = pd.read_html(table.prettify())[0]
    df.dropna(inplace = True, how="all")
    
    df['Year'] = Year
    df['Round'] = Round
    #df['Opening Rank'] = df['Opening Rank'].astype(int)
    #df['Closing Rank'] = df['Closing Rank'].astype(int)
    
    return df

In [5]:
df = scrape('2020', '6')
df

AttributeError: 'NoneType' object has no attribute 'prettify'

In [ ]:
#df['Opening Rank']= df['Opening Rank'].str.replace('P', '')
open = df.loc[(df['Seat Type']=='OPEN')& (df['Gender']=='Gender-Neutral') & (df.Quota=='AI')]
open['Opening Rank'] = open['Opening Rank'].astype(int)
open['Closing Rank'] = open['Closing Rank'].astype(int)
open

In [113]:
open = open.sort_values('Opening Rank')
open.head(40)

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
186,Indian Institute of Technology Bombay,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,2,66,2020,6
8650,"School of Planning & Architecture, New Delhi","Architecture (5 Years, Bachelor of Architecture)",AI,OPEN,Gender-Neutral,3,230,2020,6
212,Indian Institute of Technology Bombay,"Electrical Engineering (4 Years, Bachelor of T...",AI,OPEN,Gender-Neutral,25,298,2020,6
434,Indian Institute of Technology Delhi,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,31,105,2020,6
8657,"School of Planning & Architecture, New Delhi","Planning (4 Years, Bachelor of Planning)",AI,OPEN,Gender-Neutral,46,479,2020,6
1414,Indian Institute of Technology Madras,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,49,158,2020,6
8662,School of Planning & Architecture: Vijayawada,"Architecture (5 Years, Bachelor of Architecture)",AI,OPEN,Gender-Neutral,68,694,2020,6
450,Indian Institute of Technology Delhi,"Computer Science and Engineering (5 Years, Bac...",AI,OPEN,Gender-Neutral,108,189,2020,6
1255,Indian Institute of Technology Kanpur,"Computer Science and Engineering (4 Years, Bac...",AI,OPEN,Gender-Neutral,111,239,2020,6
144,Indian Institute of Technology Bombay,"BS in Mathematics (4 Years, Bachelor of Science)",AI,OPEN,Gender-Neutral,128,1567,2020,6


In [1]:
open.Institute.str.split(' ')


AttributeError: 'function' object has no attribute 'Institute'